In [8]:
#install.packages("readxl")
#install.packages("dplyr")
#install.packages("caret")
#install.packages("xgboost")
#install.packages("pROC")  # For ROC curve
#install.packages("ggplot2")  # For feature importance plotting
#install.packages("FNN")
#install.packages("keras")
install.packages("e1071")

Installation du package dans 'C:/Users/naouf/AppData/Local/R/win-library/4.4'
(car 'lib' n'est pas spécifié)



le package 'e1071' a été décompressé et les sommes MD5 ont été vérifiées avec succés

Les packages binaires téléchargés sont dans
	C:\Users\naouf\AppData\Local\Temp\RtmpcZmvpr\downloaded_packages


In [9]:
library(readxl)
library(dplyr)
library(caret)
library(xgboost)
library(pROC)
library(ggplot2)
library(FNN)
library(e1071)

## Data :

In [10]:
HighPackage <- read_xlsx('../Data/Data_HighPackage.xlsx',.name_repair = "universal")
head(HighPackage,5)

New names:
• `` -> `...1`
• `Master name` -> `Master.name`
• `Mass [kg]` -> `Mass..kg.`


...1,id,Prod_Year,Master.name,Mass..kg.
<dbl>,<dbl>,<dbl>,<chr>,<dbl>
8388,8388,1987,MASTER-87-0001,907.9739
7295,7295,1987,MASTER-87-0002,433.4347
4444,4444,1987,MASTER-87-0003,631.8340
10400,10400,1987,MASTER-87-0004,1135.6316
10387,10387,1987,MASTER-87-0005,972.7971


In [11]:
LinkLowHigh <- read_xlsx('../Data/DataLinkLowHigh.xlsx',.name_repair = "universal")
head(LinkLowHigh,5)

New names:
• `Colli name` -> `Colli.name`
• `Master-Pack` -> `Master.Pack`


Colli.name,index,Master.Pack
<chr>,<chr>,<chr>
P-85-0000,P-85-0000,MASTER-94-0181
P-85-0001,P-85-0001,MASTER-20-0147
P-85-0002,P-85-0002,MASTER-13-0254
P-85-0003,P-85-0003,MASTER-24-0232
P-85-0004,P-85-0004,MASTER-94-0238


In [12]:
LowAct <- read.csv('../Data/Data_LowAct.csv')
head(LowAct,5)

,X,Colli.name,Radionuclide,Activity
,<int>,<chr>,<chr>,<dbl>
1,0,P-85-0000,Co-60,1787374958
2,1,P-85-0001,Co-60,4594578656
3,2,P-85-0002,Co-60,904805827
4,3,P-85-0003,Co-60,983771363
5,4,P-85-0004,Co-60,558225661


In [13]:
LowCompaction <- read_xlsx('../Data/Data_LowCompaction.xlsx',.name_repair = "universal")
head(LowCompaction,5)

New names:
• `` -> `...1`
• `Colli name` -> `Colli.name`
• `Mass [kg]` -> `Mass..kg.`


...1,Colli.name,Mass..kg.,Fraction,Fraction_Split
<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,P-85-0000,67.66372771102851,0.09342582,1
2,P-85-0001,209.8018069246933,0.10003202,1
3,P-85-0002,93.65991565238585,0.12187675,1
4,P-85-0003,100.7469625771596,0.10009724,1
5,P-85-0004,67.17269270748484,0.10043900,1


In [14]:
LowLowPackage <- read_xlsx('../Data/Data_LowLowPackage.xlsx',.name_repair = "universal")
head(LowLowPackage,5)
nrow(LowLowPackage)

New names:
• `` -> `...1`
• `Unnamed: 0` -> `Unnamed..0`
• `Colli name` -> `Colli.name`
• `Mass [kg]` -> `Mass..kg.`
• `Cellulose [g]` -> `Cellulose..g.`


...1,Unnamed..0,Prod_year,Colli.name,State,Condit,Material,Mass..kg.,Mass_Concrete,Mass_Metal,Mass_Plastic,Cellulose..g.
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,64036,2010,P-10-0009,Solid,Supercompaction,Mixture,68.17947,29.75258,9.780126,28.64676,517.6531
1,64037,2010,P-10-0010,Solid,Supercompaction,Mixture,149.44717,48.67213,53.277507,47.49753,2711.3729
2,64038,2010,P-10-0011,Solid,Supercompaction,Mixture,169.42969,60.87829,8.704218,99.84718,535.0581
3,64041,2010,P-10-0014,Solid,Supercompaction,Mixture,228.48313,33.70318,117.301721,77.47822,5942.5643
4,64046,2010,P-10-0019,Solid,Supercompaction,Mixture,167.61310,86.00925,58.342785,23.26106,2940.4003


[1] 14262

In [15]:
LowPackage <- read_xlsx('../Data/Data_LowPackage.xlsx',.name_repair = "universal")
head(LowPackage,5)
nrow(LowPackage)

New names:
• `` -> `...1`
• `Colli name` -> `Colli.name`
• `Mass Pack [kg]` -> `Mass.Pack..kg.`
• `Volume [L]` -> `Volume..L.`
• `Mass [kg]` -> `Mass..kg.`
• `Act_BETA_GAMMA [Bq]` -> `Act_BETA_GAMMA..Bq.`
• `Act_ALPHA [Bq]` -> `Act_ALPHA..Bq.`
• `Dose Debit [mmSv/h]` -> `Dose.Debit..mmSv.h.`
• `Measure date` -> `Measure.date`


...1,Prod_year,Producteur,Colli.name,Send_Date,Receipt_Date,State,Condit,Material,Mass.Pack..kg.,Volume..L.,Mass..kg.,Act_BETA_GAMMA..Bq.,Act_ALPHA..Bq.,Dose.Debit..mmSv.h.,Measure.date,Acceptor
<dbl>,<dbl>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dttm>,<chr>
1,1985,Prod1,P-85-0000,1985-03-31,1985-04-03,Solid,Supercompaction,Concrete,20,200,67.66372771102851,181567355,4656133,28.64554,1985-03-31,Florian Lemoine
2,1985,Prod1,P-85-0001,1985-11-21,1985-11-21,Solid,Supercompaction,Mixture,20,200,209.8018069246933,493055528,11968931,37.84054,1985-11-23,Phillipe Migeot
3,1985,Prod1,P-85-0002,1985-11-04,1985-11-07,Solid,Supercompaction,Metal,20,200,93.65991565238585,422351683,2357030,36.97683,1985-11-05,Adrien Le Gentille
4,1985,Prod1,P-85-0003,1985-08-21,1985-08-26,Solid,Supercompaction,Mixture,20,200,100.7469625771596,610617458,2562736,12.44557,1985-08-21,Mert Güngör
5,1985,Prod1,P-85-0004,1985-11-15,1985-11-20,Solid,Supercompaction,Concrete,24,200,67.17269270748484,159259530,1454184,24.64728,1985-11-16,Adrien Le Gentille


[1] 100000

In [16]:
LowPrice <- read_xlsx('../Data/Data_LowPrice.xlsx',.name_repair = "universal")
head(LowPrice,5)

New names:
• `` -> `...1`
• `Mass [kg]` -> `Mass..kg.`
• `Price [€]` -> `Price....`


...1,Prod_year,Mass..kg.,Price....
<dbl>,<dbl>,<dbl>,<dbl>
0,1985,67.66373,4228.983
1,1985,209.80181,13112.613
2,1985,93.65992,5853.745
3,1985,100.74696,6296.685
4,1985,67.17269,4198.293


# Cleaning Process :

In [17]:
HighPackage_clean <- HighPackage[,c(-1)]
head(HighPackage_clean,5)

id,Prod_Year,Master.name,Mass..kg.
<dbl>,<dbl>,<chr>,<dbl>
8388,1987,MASTER-87-0001,907.9739
7295,1987,MASTER-87-0002,433.4347
4444,1987,MASTER-87-0003,631.8340
10400,1987,MASTER-87-0004,1135.6316
10387,1987,MASTER-87-0005,972.7971


In [18]:
colnames(LinkLowHigh)[colnames(LinkLowHigh) == "Master.Pack"] <- "Master.name"
head(LinkLowHigh,5)

Colli.name,index,Master.name
<chr>,<chr>,<chr>
P-85-0000,P-85-0000,MASTER-94-0181
P-85-0001,P-85-0001,MASTER-20-0147
P-85-0002,P-85-0002,MASTER-13-0254
P-85-0003,P-85-0003,MASTER-24-0232
P-85-0004,P-85-0004,MASTER-94-0238


In [19]:
LowAct_clean <- LowAct[,c(-1)]
# Remove rows containing either NA or NULL
LowAct_clean <- LowAct_clean[apply(LowAct_clean, 1, function(row) all(!sapply(row, is.null))) & complete.cases(LowAct_clean), ]
                                            
# Remove columns where all values are the same
LowAct_clean <- LowAct_clean[, apply(LowAct_clean, 2, function(x) length(unique(x)) > 1 | any(!is.na(x)))]
                       
# Remove rows where any numeric column has a negative value
LowAct_clean <- LowAct_clean[!apply(LowAct_clean, 1, function(row) any(row < 0 & is.numeric(row))), ]

head(LowAct_clean,5)

,Colli.name,Radionuclide,Activity
,<chr>,<chr>,<dbl>
1,P-85-0000,Co-60,1787374958
2,P-85-0001,Co-60,4594578656
3,P-85-0002,Co-60,904805827
4,P-85-0003,Co-60,983771363
5,P-85-0004,Co-60,558225661


In [38]:
LowCompaction_clean <- LowCompaction[,c(-1,-4,-5)]

# Convert character to numeric (double)
#LowCompaction_clean$Mass..kg. <- as.numeric(LowCompaction_clean$Mass..kg.)

# Remove rows containing either NA or NULL
LowCompaction_clean <- LowCompaction_clean[apply(LowCompaction_clean, 1, function(row) all(!sapply(row, is.null))) & complete.cases(LowCompaction_clean), ]
                                            
# Remove columns where all values are the same
LowCompaction_clean <- LowCompaction_clean[, apply(LowCompaction_clean, 2, function(x) length(unique(x)) > 1 | any(!is.na(x)))]
                       
# Remove rows where any numeric column has a negative value
LowCompaction_clean <- LowCompaction_clean[!apply(LowCompaction_clean, 1, function(row) any(row < 0 & is.numeric(row))), ]


head(LowCompaction_clean,5)

Colli.name,Mass..kg.
<chr>,<chr>
P-85-0000,67.66372771102851
P-85-0001,209.8018069246933
P-85-0002,93.65991565238585
P-85-0003,100.7469625771596
P-85-0004,67.17269270748484


In [21]:
#Remove the material feature because the cellulose can and should be only found In a Mixture
LowLowPackage_clean <- LowLowPackage[c("Colli.name","Condit","Mass..kg.","Cellulose..g.")]
# Remove rows containing either NA or NULL
LowLowPackage_clean <- LowLowPackage_clean[apply(LowLowPackage_clean, 1, function(row) all(!sapply(row, is.null))) & complete.cases(LowLowPackage_clean), ]
                                            
# Remove columns where all values are the same
LowLowPackage_clean <- LowLowPackage_clean[, apply(LowLowPackage_clean, 2, function(x) length(unique(x)) > 1 | any(!is.na(x)))]
                       
# Remove rows where any numeric column has a negative value
LowLowPackage_clean <- LowLowPackage_clean[!apply(LowLowPackage_clean, 1, function(row) any(row < 0 & is.numeric(row))), ]

                                                  
head(LowLowPackage_clean,5)

Colli.name,Condit,Mass..kg.,Cellulose..g.
<chr>,<chr>,<dbl>,<dbl>
P-10-0009,Supercompaction,68.17947,517.6531
P-10-0010,Supercompaction,149.44717,2711.3729
P-10-0011,Supercompaction,169.42969,535.0581
P-10-0014,Supercompaction,228.48313,5942.5643
P-10-0019,Supercompaction,167.61310,2940.4003


In [39]:
LowPackage_clean <- LowPackage[c("Colli.name", "Material", "Condit", 
                    "Volume..L.", "Mass.Pack..kg.", "Mass..kg.", "Act_BETA_GAMMA..Bq.", 
                    "Act_ALPHA..Bq.", "Dose.Debit..mmSv.h.")]
LowPackage_clean <- LowPackage_clean[LowPackage_clean$Material == "Mixture", ]

# Remove rows containing either NA or NULL
LowPackage_clean <- LowPackage_clean[apply(LowPackage_clean, 1, function(row) all(!sapply(row, is.null))) & complete.cases(LowPackage_clean), ]
                                            
# Remove columns where all values are the same
LowPackage_clean <- LowPackage_clean[, apply(LowPackage_clean, 2, function(x) length(unique(x)) > 1 | any(!is.na(x)))]
                       
# Remove rows where any numeric column has a negative value
LowPackage_clean <- LowPackage_clean[!apply(LowPackage_clean, 1, function(row) any(row < 0 & is.numeric(row))), ]

LowPackage_clean <- LowPackage_clean[, !(colnames(LowPackage_clean) %in% c("Material","Volume..L."))]

# Convert character to numeric (double)
LowPackage_clean$Mass..kg. <- as.numeric(LowPackage_clean$Mass..kg.)

head(LowPackage_clean,5)                

Colli.name,Condit,Mass.Pack..kg.,Mass..kg.,Act_BETA_GAMMA..Bq.,Act_ALPHA..Bq.,Dose.Debit..mmSv.h.
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P-85-0001,Supercompaction,20,209.8018,493055528,11968931,37.840542
P-85-0003,Supercompaction,20,100.7470,610617458,2562736,12.445572
P-85-0006,Supercompaction,20,173.1205,1157958817,1942643,8.100419
P-85-0007,Supercompaction,24,243.9984,1568401001,1304739,24.285040
P-85-0008,Supercompaction,20,249.9962,2042681809,14367017,31.966304


In [23]:
#head(LowPrice,5)
#TODO if needed

In [24]:
#Cheat sheet
# Remove rows containing either NA or NULL
#df_clean <- df[apply(df, 1, function(row) all(!sapply(row, is.null))) & complete.cases(df), ]
                                            
# Remove columns where all values are the same
#df_clean <- df[, apply(df, 2, function(x) length(unique(x)) > 1 | any(!is.na(x)))]
                       
# Remove rows where any numeric column has a negative value
#df_clean <- df[!apply(df, 1, function(row) any(row < 0 & is.numeric(row))), ]
             

# Feature Selection :

## adjusting features

In [40]:
# One-hot encode the 'Condit' column using model.matrix
one_hot_encoded <- model.matrix(~ Condit - 1, data = LowPackage_clean)
LowPackage_clean <- LowPackage_clean[, !(colnames(LowPackage_clean) == "Condit")]
LowPackage_clean_with_one_hot <- cbind(LowPackage_clean,one_hot_encoded)

head(LowPackage_clean_with_one_hot,10)
nrow(LowPackage_clean)
nrow(LowPackage_clean_with_one_hot)

,Colli.name,Mass.Pack..kg.,Mass..kg.,Act_BETA_GAMMA..Bq.,Act_ALPHA..Bq.,Dose.Debit..mmSv.h.,ConditCombustion,ConditSupercompaction
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P-85-0001,20,209.80181,493055528,11968931.0,37.8405422,0,1
2,P-85-0003,20,100.74696,610617458,2562735.9,12.4455724,0,1
3,P-85-0006,20,173.12049,1157958817,1942642.8,8.1004189,0,1
4,P-85-0007,24,243.99837,1568401001,1304738.7,24.2850402,0,1
5,P-85-0008,20,249.99620,2042681809,14367016.9,31.9663041,0,1
6,P-85-0010,20,68.30295,16239582,5453095.3,0.4266768,0,1
7,P-85-0013,20,114.54380,814416814,5398889.2,0.2127710,0,1
8,P-85-0015,20,242.81689,966467652,440029.1,3.0853596,0,1
9,P-85-0016,24,217.52070,974065960,20983153.6,25.1875075,0,1


[1] 39410

[1] 39410

In [41]:
# One-hot encode the 'Condit' column using model.matrix
one_hot_encoded <- model.matrix(~ Condit - 1, data = LowLowPackage_clean)
LowLowPackage_clean <- LowLowPackage_clean[, !(colnames(LowLowPackage_clean) == "Condit")]
LowLowPackage_clean_with_one_hot <- cbind(LowLowPackage_clean,one_hot_encoded)

#LowLowPackage_clean_with_one_hot$Cellulose..g. <- ifelse(LowLowPackage_clean_with_one_hot$Cellulose..g. > 125, 0, 1)

head(LowLowPackage_clean_with_one_hot,10)
nrow(LowLowPackage_clean)
nrow(LowLowPackage_clean_with_one_hot)

ERROR: Error in eval(predvars, data, env): objet 'Condit' introuvable


In [42]:
head(LowCompaction_clean,5)
nrow(LowCompaction_clean)


Colli.name,Mass..kg.
<chr>,<chr>
P-85-0000,67.66372771102851
P-85-0001,209.8018069246933
P-85-0002,93.65991565238585
P-85-0003,100.7469625771596
P-85-0004,67.17269270748484


[1] 86007

In [28]:
# Create a one-hot encoded matrix for the "Radionuclide" column
one_hot_encoded <- model.matrix
(~ Radionuclide - 1, data = LowAct_clean)
LowAct_clean_with_one_hot <- cbind(LowAct_clean, one_hot_encoded)
LowAct_clean_with_one_hot <- LowAct_clean_with_one_hot[, !names(LowAct_clean_with_one_hot) %in% "Radionuclide"]

# Aggregate the data by 'Colli.name', summing the one-hot encoded columns
LowAct_clean_with_one_hot <- aggregate(. ~ Colli.name, data = LowAct_clean_with_one_hot, FUN = sum)
# Remove duplicates based on all columns
#LowAct_clean_with_one_hot <- LowAct_clean_with_one_hot[!duplicated(LowAct_clean_with_one_hot), ]
head(LowAct_clean_with_one_hot,5)
nrow(LowAct_clean_with_one_hot)


,Colli.name,Activity,RadionuclideAm-241,RadionuclideC-14,RadionuclideCa-41,RadionuclideCm-244,RadionuclideCo-60,RadionuclideCs-137,RadionuclideNb-94,RadionuclideNi-59,RadionuclideNi-63,RadionuclideNp-237,RadionuclidePu-239,RadionuclidePu-240,RadionuclidePu-241,RadionuclideU-235,RadionuclideU-236,RadionuclideZr-63
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P-00-0000,8265136,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,P-00-0001,6416681,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,P-00-0002,33356731075,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,P-00-0003,42527783747,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,P-00-0004,48611514958,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


[1] 100000

In [43]:
# Get the common column names (excluding 'Colli.name')
common_cols <- intersect(names(LowPackage_clean_with_one_hot), names(LowLowPackage_clean_with_one_hot))
common_cols <- common_cols[common_cols != "Colli.name"]

model_data <- LowLowPackage_clean_with_one_hot %>%
    inner_join(LowPackage_clean_with_one_hot[, !names(LowPackage_clean_with_one_hot) %in% common_cols]) %>%
    inner_join(LowAct_clean_with_one_hot, by = "Colli.name") %>%
    inner_join(LowCompaction_clean[, !names(LowCompaction_clean) %in% "Mass..kg."], by = "Colli.name")
head(model_data,5)
nrow(model_data)

Joining with `by = join_by(Colli.name)`


,Colli.name,Mass..kg.,Cellulose..g.,ConditCombustion,ConditSupercompaction,Mass.Pack..kg.,Act_BETA_GAMMA..Bq.,Act_ALPHA..Bq.,Dose.Debit..mmSv.h.,Activity,⋯,RadionuclideNb-94,RadionuclideNi-59,RadionuclideNi-63,RadionuclideNp-237,RadionuclidePu-239,RadionuclidePu-240,RadionuclidePu-241,RadionuclideU-235,RadionuclideU-236,RadionuclideZr-63
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P-10-0009,68.17947,517.6531,0,1,24,558656040,6760695,22.466918,65556395829,⋯,1,1,1,1,1,1,1,1,1,1
2,P-10-0010,149.44717,2711.3729,0,1,20,414286579,6589755,24.824688,63898838373,⋯,1,1,1,1,1,1,1,1,1,1
3,P-10-0011,169.42969,535.0581,0,1,24,172619749,13117566,35.527515,127197036798,⋯,1,1,1,1,1,1,1,1,1,1
4,P-10-0014,228.48313,5942.5643,0,1,20,471936584,8597763,36.249804,83369887542,⋯,1,1,1,1,1,1,1,1,1,1
5,P-10-0019,167.61310,2940.4003,0,1,24,791482026,13174869,1.271811,127752683677,⋯,1,1,1,1,1,1,1,1,1,1


[1] 12395

In [44]:
# Shuffle the data
set.seed(42)  # For reproducibility
model_data <- model_data[sample(nrow(model_data)), ]

# Separate features (X) and target variable (Y)
X <- model_data[, !names(model_data) %in% c("Colli.name", "Cellulose..g.")]
Y <- model_data$Cellulose..g.

# Split into train and test sets (80% train, 20% test)
library(caret)
trainIndex <- createDataPartition(Y, p = 0.8, list = FALSE)
X_train <- X[trainIndex, ]
Y_train <- Y[trainIndex]
X_test <- X[-trainIndex, ]
Y_test <- Y[-trainIndex]

# Train an SVM model
library(e1071)

# For regression
svm_model <- svm(X_train, Y_train, type = "eps-regression", kernel = "radial")

# For classification (if Y is categorical)
# svm_model <- svm(X_train, as.factor(Y_train), type = "C-classification", kernel = "radial")

# Make predictions
predictions <- predict(svm_model, X_test)

# Evaluate model performance
# For regression
rmse_value <- sqrt(mean((predictions - Y_test)^2))
cat("RMSE:", rmse_value, "\n")

# For classification (if classification was used)
# confusion_matrix <- table(Predicted = predictions, Actual = Y_test)
# print(confusion_matrix)

# If you need to link results back to Colli.name:
result_df <- data.frame(Colli.name = model_data[-trainIndex, "Colli.name"], Predicted = predictions)

print(result_df)

Warning message in svm.default(X_train, Y_train, type = "eps-regression", kernel = "radial"):
"Variable(s) 'ConditCombustion' and 'ConditSupercompaction' and 'RadionuclideAm-241' and 'RadionuclideC-14' and 'RadionuclideCa-41' and 'RadionuclideCm-244' and 'RadionuclideCo-60' and 'RadionuclideCs-137' and 'RadionuclideNb-94' and 'RadionuclideNi-59' and 'RadionuclideNi-63' and 'RadionuclideNp-237' and 'RadionuclidePu-239' and 'RadionuclidePu-240' and 'RadionuclidePu-241' and 'RadionuclideU-235' and 'RadionuclideU-236' and 'RadionuclideZr-63' constant. Cannot scale data."


RMSE: 2448.426 
      Colli.name Predicted
1252   P-11-0797  1671.452
10289  P-21-1607  1671.452
3954   P-14-0876  1671.452
9189   P-20-0765  1671.452
12136  P-23-1702  1671.452
8491   P-19-1510  1671.452
5468   P-16-0395  1671.452
2274   P-12-1458  1671.452
11509  P-22-2465  1671.452
2818   P-13-0287  1671.452
12353  P-23-2371  1671.452
10973  P-22-0745  1671.452
11512  P-22-2470  1671.452
7140   P-18-0154  1671.452
100    P-10-0214  1671.452
2346   P-12-1595  1671.452
7349   P-18-0571  1671.452
10528  P-21-2319  1671.452
9501   P-20-1972  1671.452
6782   P-17-1673  1671.452
12281  P-23-2237  1671.452
8102   P-19-0274  1671.452
9729   P-21-0091  1671.452
1889   P-12-0299  1671.452
3310   P-13-1743  1671.452
6670   P-17-1441  1671.452
4395   P-14-2454  1671.452
10752  P-22-0322  1671.452
2910   P-13-0466  1671.452
1248   P-11-0791  1671.452
2089   P-12-0679  1671.452
3995   P-14-0955  1671.452
12154  P-23-1738  1671.452
3578   P-14-0091  1671.452
3252   P-13-1620  1671.452
3196   P-13-

In [45]:
#install.packages("randomForest")
#library(randomForest)

# Train a random forest model
rf_model <- randomForest(X_train, Y_train, ntree = 10)

# Make predictions
rf_predictions <- predict(rf_model, X_test)
print(rf_model)
# Evaluate RMSE
rf_rmse <- sqrt(mean((rf_predictions - Y_test)^2))
cat("Random Forest RMSE:", rf_rmse, "\n")


Call:
 randomForest(x = X_train, y = Y_train, ntree = 10) 
               Type of random forest: regression
                     Number of trees: 10
No. of variables tried at each split: 8

          Mean of squared residuals: 5173860
                    % Var explained: 3.8
Random Forest RMSE: 2227.925 


In [34]:
library(keras)
model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = "relu", input_shape = ncol(X)) %>%
  layer_dense(units = 1)

model %>% compile(loss = "mse", optimizer = "adam")
model %>% fit(X_train, Y_train, epochs = 100, batch_size = 32, validation_split = 0.2)


ERROR: Error: Valid installation of TensorFlow not found.

Python environments searched for 'tensorflow' package:
 C:\Users\naouf\Documents\.virtualenvs\r-tensorflow\Scripts\python.exe

Python exception encountered:
 Traceback (most recent call last):
  File "C:\Users\naouf\AppData\Local\R\win-library\4.4\reticulate\python\rpytools\loader.py", line 122, in _find_and_load_hook
    return _run_hook(name, _hook)
  File "C:\Users\naouf\AppData\Local\R\win-library\4.4\reticulate\python\rpytools\loader.py", line 96, in _run_hook
    module = hook()
  File "C:\Users\naouf\AppData\Local\R\win-library\4.4\reticulate\python\rpytools\loader.py", line 120, in _hook
    return _find_and_load(name, import_)
ModuleNotFoundError: No module named 'tensorflow'


You can install TensorFlow using the install_tensorflow() function.



In [ ]:
new_data <- LowPackage_clean_with_one_hot %>%
    inner_join(LowAct_clean_with_one_hot, by = "Colli.name") %>%
    inner_join(LowCompaction_clean[, !names(LowCompaction_clean) %in% "Mass..kg."], by = "Colli.name")

new_data <- new_data[,names(model_data[, !names(model_data) %in% "Cellulose..g."])]
min(k_df$rmse)

In [ ]:
# 1. Prepare the data by removing the 'Colli.name' column
X_new <- as.matrix(new_data[, !names(new_data) %in% c("Colli.name")])  # Remove 'Colli.name'

# 2. Perform KNN regression using the best value of k from k_df
k <- k_df$k[which.min(k_df$rmse)]  # Get the best k

# Train the KNN model on the training set (using X_train and Y_train) and predict on X_new
knn_model <- knn.reg(train = X_train, test = X_new, y = Y_train, k = k)  # For regression

# 3. Create the results by adding 'Colli.name' back
result_df <- data.frame(Colli.name = new_data$Colli.name, Predicted = knn_model$pred)

# View the result dataframe (with 'Colli.name' and predictions)
head(result_df,5)


In [ ]:
# Step 1: Merge result_df with LinkLowHigh on 'Colli.name'
merged_link_low_high <- merge(result_df, LinkLowHigh, by = 'Colli.name', all.x = TRUE)

# Step 2: Merge the above result with HighPackage_clean on 'Master-Pack'
final_merged <- merge(merged_link_low_high, HighPackage_clean, by = 'Master.name', all.x = TRUE)

# Step 3: Filter rows where all the necessary links are present
# We keep only the rows where the 'Predicted' column from result_df is not NA
final_result <- final_merged[!is.na(final_merged$Predicted), ]

# View the final merged result
head(final_result)
colnames(final_result)[colnames(final_result) == "Predicted"] <- "Cellulose.g"
write.csv(final_result, "../Output/HighLowCellulose.csv", row.names = TRUE)

In [ ]:
# Remove 'Colli.name' column
df_cleaned <- final_result[, !names(final_result) %in% "Colli.name"]

# Group by 'Master.name' and sum 'Cellulose.g'
aggregated_df <- df_cleaned %>%
  group_by(Master.name) %>%
  summarise(sum_Cellulose = sum(Cellulose.g, na.rm = TRUE))

# Calculate the total sum of Cellulose.g
total_sum <- sum(aggregated_df$sum_Cellulose)

# Add a column to flag if the sum of cellulose for each Master.name surpasses 62.5% of the total
aggregated_df$flag <- ifelse(aggregated_df$sum_Cellulose > 0.625 * total_sum, "Flagged", "Not Flagged")

# View the result
print(aggregated_df)

In [ ]:
# Count how many rows are flagged
count_flagged <- sum(aggregated_df$flag == "Flagged")

# Print the result
print(paste("Number of flagged rows:", count_flagged))